In [125]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [126]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("zh-plus/tiny-imagenet")['train']
        else:
            temp_dataset = load_dataset("zh-plus/tiny-imagenet")['valid']
        
        self.dataset = []
        for i in range(len(temp_dataset)):
            if temp_dataset[i]['label'] in [27, 28, 29]:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['image']).expand((3, -1, -1)) / 255.0
                self.dataset.append([img, torch.tensor([0])])

            elif torch.randint(0, 200, (1, )) < 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['image']).expand((3, -1, -1)) / 255.0
                self.dataset.append([img, torch.tensor([1])])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = torchvision.transforms.RandomHorizontalFlip()(image)
        image = torchvision.transforms.RandomResizedCrop((64, 64), scale=(4/5, 5/4), ratio=(4/5, 5/4))(image)
        return (image, label)

In [127]:
train_dataset = CatsDataset()
len(train_dataset)

2942

In [128]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=True
)

In [129]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [130]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Conv2d(16, 32, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Conv2d(32, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(64, 2)
).to(dev)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [131]:
for epoch in range(100):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1 Loss: 0.6906811644633611
Epoch 2 Loss: 0.6727351347605387
Epoch 3 Loss: 0.6353405018647512
Epoch 4 Loss: 0.596834272146225
Epoch 5 Loss: 0.5716267128785452
Epoch 6 Loss: 0.5597953250010809
Epoch 7 Loss: 0.5613003025452296
Epoch 8 Loss: 0.5604762683312098
Epoch 9 Loss: 0.5618953530987104
Epoch 10 Loss: 0.5479808151721954
Epoch 11 Loss: 0.5520919859409332
Epoch 12 Loss: 0.5400112718343735
Epoch 13 Loss: 0.5486462861299515
Epoch 14 Loss: 0.5329140002528826
Epoch 15 Loss: 0.5428773512442907
Epoch 16 Loss: 0.5365043183167776
Epoch 17 Loss: 0.5237851167718569
Epoch 18 Loss: 0.5268584787845612
Epoch 19 Loss: 0.5223747367660204
Epoch 20 Loss: 0.5147808541854223
Epoch 21 Loss: 0.5220219567418098
Epoch 22 Loss: 0.5204466407497724
Epoch 23 Loss: 0.5177418912450472
Epoch 24 Loss: 0.5181470091144244
Epoch 25 Loss: 0.5142489224672318
Epoch 26 Loss: 0.5052246699730555
Epoch 27 Loss: 0.4934556633234024
Epoch 28 Loss: 0.493755670885245
Epoch 29 Loss: 0.49155181894699734
Epoch 30 Loss: 0.4935596

In [132]:
test_dataset = CatsDataset(train=False)

In [133]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)

In [134]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.7880, device='mps:0')
